In [70]:
pip install tensorflow


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [71]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras import layers, models

train = pd.read_csv('train_df.csv')
test = pd.read_csv('test_df.csv')

validation_subjects = {"co2a0000377", "co2a0000364", "co2c0000342", "co2c0000345"}
val = test[test['subject'].isin(validation_subjects)].drop(columns=['sample', 'match', 'err', 'object'])
test = test[~test['subject'].isin(validation_subjects)]
train = train[~train['subject'].isin(validation_subjects)]

train = train.drop(columns=['sample', 'match', 'err', 'object'])
test = test.drop(columns=['sample', 'match', 'err', 'object'])

X_train = train.drop(columns=['alcoholic', 'subject'])
y_train = train[['alcoholic']]

X_test = test.drop(columns=['alcoholic', 'subject'])
y_test = test[['alcoholic']]

X_val = val.drop(columns=['alcoholic', 'subject'])
y_val = val[['alcoholic']]

# normalize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

# model layers
model = models.Sequential()

model.add(layers.Conv1D(64, 3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(layers.MaxPooling1D(2))

model.add(layers.Conv1D(128, 3, activation='relu'))
model.add(layers.MaxPooling1D(2))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

/Users/dpark_23/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [72]:
# train
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# evaluate
test_metrics = model.evaluate(X_test, y_test)

# prints
test_accuracy = test_metrics[1]
print(f'Test Accuracy: {test_accuracy}')

# class report
y_pred = (model.predict(X_test) > 0.5)
print(classification_report(y_test, y_pred))

Epoch 1/10
3840/3840 ━━━━━━━━━━━━━━━━━━━━ 22s 5ms/step - accuracy: 0.8012 - loss: 0.4055 - precision_6: 0.8057 - recall_6: 0.7910 - val_accuracy: 0.8198 - val_loss: 0.4972 - val_precision_6: 0.8713 - val_recall_6: 0.7505
Epoch 2/10
3840/3840 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - accuracy: 0.9247 - loss: 0.1845 - precision_6: 0.9240 - recall_6: 0.9258 - val_accuracy: 0.8480 - val_loss: 0.4725 - val_precision_6: 0.8619 - val_recall_6: 0.8288
Epoch 3/10
3840/3840 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - accuracy: 0.9525 - loss: 0.1201 - precision_6: 0.9512 - recall_6: 0.9541 - val_accuracy: 0.8559 - val_loss: 0.5083 - val_precision_6: 0.8778 - val_recall_6: 0.8270
Epoch 4/10
3840/3840 ━━━━━━━━━━━━━━━━━━━━ 22s 6ms/step - accuracy: 0.9644 - loss: 0.0906 - precision_6: 0.9633 - recall_6: 0.9656 - val_accuracy: 0.8646 - val_loss: 0.4373 - val_precision_6: 0.8568 - val_recall_6: 0.8755
Epoch 5/10
3840/3840 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - accuracy: 0.9734 - loss: 0.0690 - precision_6: 0.9729 - rec

In [74]:
# our test metrics
test_metrics = model.evaluate(X_test, y_test)
test_loss = test_metrics[0]
test_acc = test_metrics[1]

print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_acc}')

3840/3840 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.8730 - loss: 0.5729 - precision_6: 0.8287 - recall_6: 0.7885
Test Loss: 0.6492851376533508
Test Accuracy: 0.8727864623069763


In [75]:
# our validation metrics
y_val_pred = (model.predict(X_val) > 0.5)
val_metrics = model.evaluate(X_val, y_val)
val_loss = val_metrics[0]
val_acc = val_metrics[1]

print(f'Val Loss: {val_loss}')
print(f'Val Accuracy: {val_acc}')

960/960 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
960/960 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5455 - loss: 1.9766 - precision_6: 0.9440 - recall_6: 0.4980
Val Loss: 1.3376696109771729
Val Accuracy: 0.7036458253860474


In [76]:
# test classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.86      0.89      0.88     61440
        True       0.89      0.85      0.87     61440

    accuracy                           0.87    122880
   macro avg       0.87      0.87      0.87    122880
weighted avg       0.87      0.87      0.87    122880



In [77]:
# validation classification report
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

       False       0.66      0.86      0.74     15360
        True       0.80      0.55      0.65     15360

    accuracy                           0.70     30720
   macro avg       0.73      0.70      0.70     30720
weighted avg       0.73      0.70      0.70     30720

